# GPTChallenge: diagnóstico a partir de HCE

Vamos a trabajar con el corpus CodEsp (textos de historial clínico etiquetados con sus códigos CIE-10 Diagnóstico)

In [2]:
import pandas as pd
import os, re
import numpy as np

pd.options.display.max_colwidth = None

In [3]:
#los códigos están en un TSV con un código por línea
train_diag = pd.read_csv("data/train/train.tsv", sep="\t", header=None, names=["archivo", "codigo"])
train_diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8316 entries, 0 to 8315
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   archivo  8316 non-null   object
 1   codigo   8316 non-null   object
dtypes: object(2)
memory usage: 130.1+ KB


In [4]:
train_diag['codigo'].value_counts()

codigo
r52        163
r69        150
r50.9      142
i10        116
r59.9       95
          ... 
d37.030      1
c79.71       1
g25.0        1
l76.3        1
n81.2        1
Name: count, Length: 2194, dtype: int64

In [5]:
#cogemos la categoría superior de cada código y las agrupamos
train_diag['cat'] = train_diag['codigo'].str.extract(r'(\w\d\d)')
print(train_diag['cat'].value_counts())
train_diag['cat'].nunique()

cat
r52    163
r10    163
r59    160
r69    150
r50    144
      ... 
c31      1
d62      1
s53      1
s34      1
n81      1
Name: count, Length: 918, dtype: int64


918

In [6]:
categories=train_diag['cat'].value_counts()[:10]
top_categorias = categories.index.to_list()
print(top_categorias)

['r52', 'r10', 'r59', 'r69', 'r50', 'r60', 'i10', 'r11', 'n28', 'd49']


In [7]:
#seleccionamos sólo las etiquetas de este subconjunto
train_diag = train_diag[np.isin(train_diag['cat'], top_categorias)]

In [8]:
#cargamos los dos conjuntos de train
path = 'data/train/text_files/'

corpus = []
for f in [f for f in os.listdir(path) if f.endswith('.txt')]:
    with open(os.path.join(path, f), encoding="utf8") as text:
        texto = text.read()
    #buscamos códigos
    file = f[:-4]
    codigos = train_diag.query('archivo==@file')['cat'].to_list()
    codigos = list(set(codigos))
    if codigos:
        corpus.append({
            'archivo': file,
            'texto': texto,
            'codigos': codigos
        })
    
df_train = pd.DataFrame(corpus).set_index('archivo')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, S0004-06142005000700014-1 to S2340-98942015000100005-1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   texto    562 non-null    object
 1   codigos  562 non-null    object
dtypes: object(2)
memory usage: 13.2+ KB


In [9]:
df_train.sample(3)

texto  \
archivo                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## Cargar los textos del conjunto de test

In [10]:
#los códigos están en un TSV con un código por línea
test_diag = pd.read_csv("data/test/test.tsv", sep="\t", header=None, names=['archivo'])
test_diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   archivo  192 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


In [11]:
path = 'data/test/text_files/'

# Lista para almacenar los textos y sus códigos correspondientes
corpus = []

# Itera sobre los archivos de texto en el directorio
for f in [f for f in os.listdir(path) if f.endswith('.txt')]:
    with open(os.path.join(path, f), encoding="utf8") as text:
        texto = text.read()
    
    # Extrae el código del nombre del archivo
    file = f[:-4]
    
    # Agrega el texto al corpus
    corpus.append({
        'archivo': file,
        'texto': texto
    })

# Crea un DataFrame a partir del corpus
df_test = pd.DataFrame(corpus).set_index('archivo')

In [12]:
df_test.sample(3)

,texto
archivo,
S0004-06142006000700011-1,"Presentamos el caso de un varón de 61 años diagnosticado, intervenido y tratado por un adenocarcinoma rectal desde 18 meses antes. Consultó por aumento del tamaño peneano, doloroso, junto a la aparición de unas lesiones eritematosas, excrecentes y sangrantes al roce en la piel prepucial. Simultáneamente se apreciaba a la exploración linfedema hipogástrico, de la piel peneana y del escroto.\n\nSe procedió a su exéresis mediante circuncisión. El estudio anatomopatológico informó de metástasis subcutáneas prepuciales secundarias a adenocarcinoma intestinal. En la imagen macromicroscópica se apreciaba infiltración tumoral del epitelio de revestimiento plano escamoso dando lugar a una excrecencia carcinomatosa. Destacaba el escaso componente inflamatorio. A gran aumento se observa que el tumor infiltrante es un adenocarcinoma moderadamente diferenciado, de alto grado y con elevado índice mitótico.\n\nA pesar del tratamiento oncológico recibido el paciente falleció a los 4 meses del diagnóstico.\n\n"
S1130-63432013000500014-1,"Varón de 55 años diagnosticado en 2006 de LLC que recibió tratamiento inmunoquimioterápico en 2008 con Rituximab-Ciclofosfamida-Fludarabina alcanzando respuesta parcial. En 2011 se evidenció progresión de enfermedad e inició tratamiento de segunda línea con Rituximab 375mg/m2 día 1 del ciclo y Bendamustina 90 mg/m2 días 1 y 2 cada 28 días. Transcurridas 4 horas de la administración del día 1 del segundo ciclo inmunoquimioterápico, el paciente acudió a Urgencias presentando sensación distérmica, fiebre de 38oC y reacción urticariforme con prurito y eritrodermia descamativa en más del 50% de la superficie corporal que persistió 15 días. Se decidió la realización de pruebas cutáneas con bendamustina ante la posibilidad de que se tratase de una reacción alérgica a dicho citostático. La solución madre utilizada fue bendamustina 2,5mg/ml, realizándose diluciones 1/10, 1/100, 1/1000 y 1/10000 para administración intradérmica.\nDada la buena tolerancia previa al rituximab durante 6 ciclos, se asoció la reacción cutánea a la bendamustina. La toxicodermia fue clasificada de grado 4 según la Common Toxicity Criteria (versión 4.0)9 y notificada mediante la tarjeta amarilla al Sistema Español de Farmacovigilancia. Las pruebas cutáneas realizadas resultaron negativas para todas las diluciones. Sin embargo, tras la valoración del beneficio/riesgo entre la gravedad de la toxicodermia y el potencial beneficio del tratamiento con bendamustina, se decidió elaborar un PD para la administración de los siguientes ciclos de bendamustina.\nEl PD consistió en 12 etapas para la administración de la dosis total del fármaco (186,3mg). El protocolo consistió en ir aumentando de forma progresiva la velocidad de infusión y la concentración del fármaco, hasta alcanzar la dosis terapéutica total. Se prepararon 3 soluciones A, B, y C de las diluciones 1/100, 1/10 y 1/1 respectivamente. Cada una de las soluciones se utilizó durante cuatro pasos, incrementando la velocidad de infusión cada quince minutos, de manera que con cada paso se administraba el doble que en el paso anterior. En el último paso se mantuvo la velocidad constante hasta completar la administración de la dosis total definida. En la tabla 1 se resumen las soluciones y el esquema de administración del protocolo de desensibilización.\n\nLa administración se realizó en la planta de Hematología bajo supervisión médica, con una duración total de 5 horas y 37 minutos. El paciente recibió 6 ciclos de inmunoquimioterapia con bendamustina mediante dicho PD con discreto prurito cutáneo pero sin complicaciones graves.\n\n"
S0365-66912012000200003-1,"Mujer de 66 años de edad remitida a nuestro Servicio por clínica de uveítis en ojo derecho desde hacía un año. La paciente refería disminución de visión en dicho ojo pero no contaba miodesopsias, ni metamorfopsias. Entre los antecedentes médicos destacaba que hace doce años sufrió una hemorragia intrac

## Binarizar las etiquetas

In [13]:
# para entrenar un clasificador multi-etiqueta generamos una matriz binaria de las etiquetas
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['codigos'])

#Guardamos las clases utilizadas en el conjunto de train
clases = mlb.classes_
num_classes = clases.shape
print(num_classes[0])

10


## Procesamiento del lenguaje natural

In [14]:
import utils

df_train_clean = utils.clean_corpus(df_train, column_name='texto')
df_test_clean = utils.clean_corpus(df_test, column_name='texto')

In [15]:
# Hacerlo solo la primera vez

#import nltk
#nltk.download('punkt')

In [16]:
#Hay un vocabulario demasiado grande como para representarlo en una BoW

utils.plot_bow_heatmap(df_train_clean, column_name='texto', n_words=10, n_grams=1)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\aitoo/nltk_data'
    - 'c:\\Users\\aitoo\\anaconda3\\nltk_data'
    - 'c:\\Users\\aitoo\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\aitoo\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\aitoo\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


## Modelos

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


In [18]:
# Vectorizar el texto utilizando TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train_vectorized = vectorizer.fit_transform(df_train_clean['texto'])

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_vectorized, y_train, test_size=0.2, random_state=42)

In [19]:
model = Pipeline([
    ('clf', OneVsRestClassifier(RandomForestClassifier()))
])

In [20]:
# Entrenar el modelo con los datos de entrenamiento y validar con los datos de validación
model.fit(X_train_split, y_train_split)

# Realizar predicciones en el conjunto de validación
y_val_pred = model.predict(X_val_split)

# Calcular la precisión y F1-score en el conjunto de validación
accuracy_val = accuracy_score(y_val_split, y_val_pred)
f1_val = f1_score(y_val_split, y_val_pred, average='micro')

print("Validation Accuracy:", accuracy_val)
print("Validation F1-score:", f1_val)

Validation Accuracy: 0.05309734513274336
Validation F1-score: 0.29007633587786263


In [21]:
from xgboost import XGBClassifier

In [22]:
model2 = Pipeline([
    ('clf', OneVsRestClassifier(XGBClassifier()))
])

In [23]:
# Entrenar el modelo con los datos de entrenamiento y validar con los datos de validación
model2.fit(X_train_split, y_train_split)

# Realizar predicciones en el conjunto de validación
y_val_pred = model2.predict(X_val_split)

# Calcular la precisión y F1-score en el conjunto de validación
accuracy_val = accuracy_score(y_val_split, y_val_pred)
f1_val = f1_score(y_val_split, y_val_pred, average='micro')

print("Validation Accuracy:", accuracy_val)
print("Validation F1-score:", f1_val)

Validation Accuracy: 0.4336283185840708
Validation F1-score: 0.8115942028985508


In [24]:
from sklearn.ensemble import AdaBoostClassifier

In [25]:
model3 = Pipeline([
    ('clf', OneVsRestClassifier(AdaBoostClassifier()))
])

In [26]:
# Entrenar el modelo con los datos de entrenamiento y validar con los datos de validación
model3.fit(X_train_split, y_train_split)

# Realizar predicciones en el conjunto de validación
y_val_pred = model3.predict(X_val_split)

# Calcular la precisión y F1-score en el conjunto de validación
accuracy_val = accuracy_score(y_val_split, y_val_pred)
f1_val = f1_score(y_val_split, y_val_pred, average='micro')

print("Validation Accuracy:", accuracy_val)
print("Validation F1-score:", f1_val)

Validation Accuracy: 0.4424778761061947
Validation F1-score: 0.815347721822542


### Modelo huggingface

In [48]:
print(X_train_split)

  (0, 5987)	0.07498257545234702
  (0, 9319)	0.22494772635704105
  (0, 8339)	0.07498257545234702
  (0, 6005)	0.07498257545234702
  (0, 6069)	0.07498257545234702
  (0, 7915)	0.07498257545234702
  (0, 583)	0.07040392912231846
  (0, 581)	0.07040392912231846
  (0, 4697)	0.22494772635704105
  (0, 6412)	0.07040392912231846
  (0, 9972)	0.07498257545234702
  (0, 2206)	0.14996515090469403
  (0, 6238)	0.14996515090469403
  (0, 9880)	0.14996515090469403
  (0, 3840)	0.22494772635704105
  (0, 219)	0.14996515090469403
  (0, 6768)	0.07040392912231846
  (0, 6793)	0.07040392912231846
  (0, 5025)	0.06257668152202976
  (0, 5392)	0.0646355169438857
  (0, 2424)	0.06257668152202976
  (0, 2098)	0.057998035192001224
  (0, 3509)	0.07040392912231846
  (0, 9878)	0.06715532785205831
  (0, 3498)	0.06257668152202976
  :	:
  (448, 4662)	0.06211201016199126
  (448, 5081)	0.10235397862668232
  (448, 1790)	0.06278723791806261
  (448, 9720)	0.038172786965036266
  (448, 731)	0.03600007852990442
  (448, 9222)	0.05737653203

## Guardar predicciones de Test